Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [36]:
from random import random, choices, randint, choice, uniform
from dataclasses import dataclass
from copy import copy
import matplotlib
import matplotlib.pyplot as plt

import lab9_lib

In [37]:
OFFSPRING_SIZE = 30
NUM_LOCI = 1000
POPULATION_SIZE = 100

In [38]:
@dataclass
class Individual:
    fitness: float
    genotype: list[bool]

def select_individual(pop):
    if (POPULATION_SIZE > 5):
        index = randint(1, 5)
    else:
        index = randint(1, POPULATION_SIZE-1)
    return pop[index]

def select_random_ind(pop):
    return choice(pop)

def select_best(pop):
    return pop[0]

def select_worse(pop):
    return pop[POPULATION_SIZE-1]

def mutate(ind):
    offspring = copy(ind)
    pos = randint(0, NUM_LOCI-1)
    if (offspring.genotype[pos] == 1):
        offspring.genotype[pos] = 0
    else:
        offspring.genotype[pos] = 1
    offspring.fitness = None
    return offspring

def invert(ind):
    for i in ind.genotype:
        if i == 1:
            i = 0
        else:
            i = 1
    return ind

def crossover(ind1: Individual, ind2: Individual, max_fitness) -> Individual:
    if (max_fitness == 1):
        return ind2
    copy_ind2 = copy(ind2.genotype)
    dim_one = randint(1, int(NUM_LOCI*(1-max_fitness)))
    dim_two = 0
    dim_three = 0
    piece_two = []
    piece_three = []
    cut_point_one = randint(0, len(copy_ind2)-1-dim_one)
    piece_one = copy_ind2[cut_point_one:cut_point_one+dim_one]
    del copy_ind2[cut_point_one:cut_point_one+dim_one]
    if (max_fitness >= 0.6):
        dim_two = randint(1, int(NUM_LOCI*(1-max_fitness)))
        cut_point_two = randint(0, len(copy_ind2)-1-dim_two)
        piece_two = copy_ind2[cut_point_two:cut_point_two+dim_two]
        del copy_ind2[cut_point_two:cut_point_two+dim_two]
        if (max_fitness >= 0.9):
            dim_three = randint(1, int(NUM_LOCI*(1-max_fitness)))
            cut_point_three = randint(0, len(copy_ind2)-1-dim_three)
            piece_three = copy_ind2[cut_point_three:cut_point_three+dim_three]
            del copy_ind2[cut_point_three:cut_point_three+dim_three]
    cut_point_ind1 = randint(0, NUM_LOCI-1-dim_one-dim_two-dim_three)
    offspring = Individual(fitness=None, genotype=ind1.genotype[:cut_point_ind1] + piece_one + piece_two + piece_three + ind1.genotype[cut_point_ind1+dim_one+dim_two+dim_three:])
    assert len(offspring.genotype) == NUM_LOCI
    return offspring

In [39]:
def run_code(population, fitness):
    for i in population:
        i.fitness = fitness(i.genotype)

    population.sort(key=lambda i: i.fitness, reverse=True)

    x=[]
    y=[]
    y2=[]

    while (population[0].fitness < 1 and fitness.calls<15000):
        offspring = list()
        for counter in range(OFFSPRING_SIZE):
            MP = uniform(0.01, 0.10)
            y2.append(population[0].fitness)
            if (population[0].fitness < 0.5):
                p1 = select_worse(population)
                o = invert(p1)
                MP = 1
            elif (population[0].fitness < 0.99):
                p1 = select_individual(population)
            else:
                p1 = select_random_ind (population)
            if random() < MP :
                o = mutate(p1)
            else:
                p2 = select_best(population)
                o = crossover(p1, p2, population[0].fitness)
            o.fitness = fitness(o.genotype)
            offspring.append(o)
            x.append(fitness.calls)
            y.append(o.fitness)
            
        population.extend(offspring)
        population.sort(key=lambda i: i.fitness, reverse=True)
        population = population[:POPULATION_SIZE]
        
    print("N. CALLS:", (fitness.calls))
    print (f"POPULATION - Fitness: {population[0].fitness:.2%} - {''.join(str(g) for g in population[0].genotype)}")
    %matplotlib qt
    plt.plot(x, y, color='red')
    plt.plot(x, y2, color='green')
    plt.show()

In [40]:
population = [
    Individual(
        genotype=choices ([0, 1], k=NUM_LOCI),
        fitness=None,
    )
    for _ in range(POPULATION_SIZE)
]

population2 = copy(population)
population5 = copy(population)
population10 = copy(population)

fitness1 = lab9_lib.make_problem(1)
fitness2 = lab9_lib.make_problem(2)
fitness5 = lab9_lib.make_problem(5)
fitness10 = lab9_lib.make_problem(10)

run_code(population, fitness1)
run_code(population2, fitness2)
run_code(population5, fitness5)
run_code(population10, fitness10)


N. CALLS: 5380
POPULATION - Fitness: 100.00% - 11111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111